In [1]:
import pandas as pd

In [2]:
import constants
import constraints

from datagolf import DataGolf
from fileman import fileman
from picklejar import PickleJar

In [3]:
DataGolf.rankings().head(10)

,name,dg-rank,skill-estimate,owgr-rank,tour
0,Rory McIlroy,1,2.378032,3,PGA
1,Xander Schauffele,2,2.049308,6,PGA
2,Scottie Scheffler,3,2.048000,1,PGA
3,Patrick Cantlay,4,2.014641,4,PGA
4,Cameron Smith,5,1.954687,2,PGA
5,Matthew Fitzpatrick,6,1.858107,10,PGA
6,Jon Rahm,7,1.796578,5,PGA
7,Justin Thomas,8,1.724348,7,PGA
8,Shane Lowry,9,1.700715,23,PGA
9,Will Zalatoris,10,1.616089,13,PGA


In [ ]:
constants.pandas_settings()
urls = {
    'skills-decomp': 'https://feeds.datagolf.com/preds/player-decompositions?tour={tour}&file_format={file_format}&key=6a626b6c312c0d33cfe157d614b5',
    'dfs-projections': 'https://feeds.datagolf.com/preds/fantasy-projection-defaults?tour={tour}&site={site}&slate={slate}&file_format={file_format}&key=6a626b6c312c0d33cfe157d614b5'
}

In [ ]:
skillsd=DataGolf.skills() #.head(10)
skillsr=DataGolf.skills(decomp=False)

master=skillsd.merge(skillsr)
order=list(master.columns)
master['delta']=master['final']/master['baseline']

master = master.loc[:, order[:3]+['delta']+order[3:]]

In [ ]:
master.head(10)

In [ ]:
def top_thresh(skill, num):
    return master.loc[master[skill]>=master[skill].describe(percentiles=(0.25, 0.33, 0.5, 0.66, 0.75, 0.82))[f'{num}%'], 'name'].values.tolist()

In [ ]:
threshold = 82

In [ ]:
def cycle():
    counter=top_thresh('delta',threshold)
    for col in list(filter(lambda x: x not in ('sg-putt', 'drv-dist'), list(master.columns)[4:])):
        counter+=top_thresh(col,threshold)
        
    count=dict()
    for name in counter:
        count[name] = count.get(name,0)+1
        
    return dict(sorted(count.items(), key=lambda x: x[1], reverse=True))

In [ ]:
results=cycle()
list(results.keys())
# results

In [ ]:
len(results)

{'last_updated': '2022-07-18 12:44:24 UTC',
 'notes': 'anyone ranked outside top 500 in OWGR appear as 501',
 'rankings': [{'am': 0,
   'country': 'NIR',
   'datagolf_rank': 1,
   'dg_id': 10091,
   'dg_skill_estimate': 2.3780316944836204,
   'owgr_rank': 3,
   'player_name': 'McIlroy, Rory',
   'primary_tour': 'PGA'},
  {'am': 0,
   'country': 'USA',
   'datagolf_rank': 2,
   'dg_id': 19895,
   'dg_skill_estimate': 2.04930840220275,
   'owgr_rank': 6,
   'player_name': 'Schauffele, Xander',
   'primary_tour': 'PGA'},
  {'am': 0,
   'country': 'USA',
   'datagolf_rank': 3,
   'dg_id': 18417,
   'dg_skill_estimate': 2.0480000341951903,
   'owgr_rank': 1,
   'player_name': 'Scheffler, Scottie',
   'primary_tour': 'PGA'},
  {'am': 0,
   'country': 'USA',
   'datagolf_rank': 4,
   'dg_id': 15466,
   'dg_skill_estimate': 2.01464050455214,
   'owgr_rank': 4,
   'player_name': 'Cantlay, Patrick',
   'primary_tour': 'PGA'},
  {'am': 0,
   'country': 'AUS',
   'datagolf_rank': 5,
   'dg_id': 15

In [ ]:
datagolf.load_dfs().head(10)

In [ ]:
datagolf.load_baselines() #.head(25)